In [8]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
from pypfopt import expected_returns, risk_models, EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation

# Define the ticker symbols of the companies you're interested in
ticker_symbols = ['AAPL', 'MSFT', 'GOOGL']  # Add more if needed

# Download historical stock price data
start_date = '2010-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

stock_data = yf.download(ticker_symbols, start=start_date, end=end_date)['Adj Close']

# Calculate annual returns for each company
annual_returns = expected_returns.mean_historical_return(stock_data)

# Calculate the covariance matrix
cov_matrix = risk_models.sample_cov(stock_data)

# Perform mean-variance optimization to get the efficient frontier
ef = EfficientFrontier(annual_returns, cov_matrix)
weights = ef.max_sharpe()

# Get the expected returns and volatility of the optimized portfolio
expected_return = ef.portfolio_performance()[0]
volatility = ef.portfolio_performance()[1]

# Assume initial investment of 100000 rupees
initial_investment = 100000

# Create empty lists to store results for each year
annual_returns_df = pd.DataFrame(columns=ticker_symbols)
portfolio_balances = []
return_percentages = []

# Calculate for each year
for year, annual_prices in stock_data.resample('Y').ffill().iterrows():
    # Calculate annual returns for each stock
    annual_returns = annual_prices.pct_change().mean() * 100
    annual_returns_df.loc[year.year] = annual_returns
    
    # Calculate portfolio balance for the year
    weights_array = np.array([weights[ticker] for ticker in ticker_symbols])
    portfolio_balance = np.dot(annual_prices.values, weights_array)
    portfolio_balances.append(portfolio_balance)  # Do not add initial investment here
    
    # Calculate return percentage for the year
    previous_balance = portfolio_balances[-2] if len(portfolio_balances) >= 2 else initial_investment
    return_percentage = ((portfolio_balance - initial_investment) / initial_investment) * 100
    return_percentages.append(return_percentage)

# Create DataFrame for annual returns
annual_returns_df.index.name = 'Year'

# Create DataFrame for balance and return percentages
balance_return_df = pd.DataFrame({
    'Portfolio Balance (Rs)': portfolio_balances,
    'Return Percentage (%)': return_percentages
}, index=stock_data.resample('Y').ffill().index.year)

print("\nAnnual Returns for Individual Stocks:")
print(annual_returns_df)

print("\nBalance and Return for Every Year:")
print(balance_return_df)


[*********************100%%**********************]  3 of 3 completed


Annual Returns for Individual Stocks:
           AAPL       MSFT      GOOGL
Year                                 
2010  48.761961  48.761961  48.761961
2011  29.709794  29.709794  29.709794
2012  16.071254  16.071254  16.071254
2013  35.989535  35.989535  35.989535
2014  29.361797  29.361797  29.361797
2015  44.545705  44.545705  44.545705
2016  45.078115  45.078115  45.078115
2017  41.569435  41.569435  41.569435
2018  61.237996  61.237996  61.237996
2019  60.127380  60.127380  60.127380
2020  57.005915  57.005915  57.005915
2021  55.097900  55.097900  55.097900
2022  68.649280  68.649280  68.649280
2023  70.673344  70.673344  70.673344
2024  89.932801  89.932801  89.932801

Balance and Return for Every Year:
      Portfolio Balance (Rs)  Return Percentage (%)
Date                                               
2010               12.070123             -99.987930
2011               14.021850             -99.985978
2012               16.899318             -99.983101
2013               